[<img src='https://lh3.googleusercontent.com/-tNe1vwwd_w4/VZ_m9E44C7I/AAAAAAAAABM/5yqhpSyYcCUzwHi-ti13MwovCb_AUD_zgCJkCGAYYCw/w256-h86-n-no/Submarineering.png'>](http://twitter.com/submarineering?lang=en)

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [4]:
train = pd.read_csv('../input/train.tsv', sep='\t')
test = pd.read_csv('../input/test.tsv', sep='\t', quoting=3, error_bad_lines=False)

#train.info()

#handel missing values:

def missing_val(dataset):
    dataset['category_name'].fillna('missing', inplace=True)
    dataset['brand_name'].fillna('missing', inplace=True)
    dataset['item_description'].fillna('missing', inplace=True)
    return dataset
    
train = missing_val(train)
test = missing_val(test)

#split the category columns in sub columns:

def split_catg_name(row):
    try:
        text = row
        text1, text2, text3 = text.split('/')
        return text1, text2, text3
    except:
        return 'no label', 'no label', 'no label'
    
train['gen_cat'], train['sub_cat1'], train['sub_cat2'] = zip(*train['category_name'].apply(lambda x : split_catg_name(x)))
test['gen_cat'], test['sub_cat1'], test['sub_cat2'] = zip(*test['category_name'].apply(lambda x : split_catg_name(x)))

#drop category_name
train.drop('category_name', axis=1, inplace=True)
test.drop('category_name', axis=1, inplace=True)

#use label encoder on categorical data:
from sklearn.preprocessing import LabelEncoder

def lab_encoder(dataset):
    le = LabelEncoder()
    dataset['gen_cat'] = le.fit_transform(dataset['gen_cat'])
    dataset['sub_cat1'] = le.fit_transform(dataset['sub_cat1'])
    dataset['sub_cat2'] = le.fit_transform(dataset['sub_cat2'])
    dataset['brand_name'] = le.fit_transform(dataset['brand_name'])
    dataset['name'] = le.fit_transform(dataset['name'])
    return dataset

train = lab_encoder(train)
test = lab_encoder(test)

#drop item_description from test and train datasets:

train.drop('item_description', axis=1, inplace=True)
test.drop('item_description', axis=1, inplace=True)

#split datasets in train and test sets:
from sklearn.cross_validation import train_test_split
train_X, test_X, train_y, test_y = train_test_split(train.drop('price', axis=1), train['price'], test_size = 0.2)

#apply Random Forest regression algorithm:
from sklearn.ensemble import RandomForestRegressor
rand_reg2 = RandomForestRegressor(n_estimators=10)
rand_reg2.fit(train_X, train_y)
rand_reg2.score(train_X, train_y)

pred = rand_reg2.predict(test)
prediction = pd.DataFrame({'test_id': test['test_id'],
                          'price':pred})


#sub = pd.read_csv('../input/sample_submission.csv')
prediction.to_csv('Submission1.csv', index=False)